In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import string
df = pd.read_json("Magazine_Subscriptions.json", lines=True)
df.to_csv()
df["asin"] = df["asin"].apply(str)
df['reviewText'] = df['reviewText'].str.lower()
df['reviewText'] = df['reviewText'].str.replace('[^\w\s]','',regex=True)
df['reviewText'] = df['reviewText'].fillna("NoReview")
df_reviews = df[["reviewerID","asin","reviewText"]]
df_reviews.head()

,reviewerID,asin,reviewText
0,AH2IFH762VY5U,B00005N7P0,for computer enthusiast maxpc is a welcome sig...
1,AOSFI0JEYU4XM,B00005N7P0,thank god this is not a ziff davis publication...
2,A3JPFWKS83R49V,B00005N7OJ,antiques magazine is a publication made for an...
3,A19FKU6JZQ2ECJ,B00005N7OJ,this beautiful magazine is in itself a work of...
4,A25MDGOMZ2GALN,B00005N7P0,a great read every issue


In [2]:
reviewCounts = pd.Series([y for x in df_reviews.values.flatten() for y in x.split()]).value_counts()
reviewCounts.head(50)

the             174797
i               121482
and             115459
to              108471
a                93202
of               75283
it               70257
magazine         67653
is               60526
for              59388
this             55926
in               45258
that             39275
my               39237
on               31389
have             29855
you              28464
not              27018
are              26845
but              25317
with             25273
great            22966
as               22173
was              20719
articles         17815
they             17512
be               17410
love             16717
like             16553
subscription     16260
read             16061
its              15940
so               15919
good             15408
all              14830
issue            14541
about            14455
or               14078
one              14023
has              13600
if               13288
me               12726
more             12559
at         

In [3]:
input = "This magazine is great, and the stories are very interesting."
input = input.lower()
input = input.translate(str.maketrans('', '', string.punctuation))
input

'this magazine is great and the stories are very interesting'

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
df_Sim = df_reviews
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Vincent/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
input_set = set(word_tokenize(input))
df_Sim['tokenReview'] = df_Sim['reviewText'].apply(word_tokenize).apply(set) 
df_Sim.head()

,reviewerID,asin,reviewText,tokenReview
0,AH2IFH762VY5U,B00005N7P0,for computer enthusiast maxpc is a welcome sig...,"{own, wait, than, blessed, info, consider, rea..."
1,AOSFI0JEYU4XM,B00005N7P0,thank god this is not a ziff davis publication...,"{davis, time, something, if, bad, actually, go..."
2,A3JPFWKS83R49V,B00005N7OJ,antiques magazine is a publication made for an...,"{means, certainly, like, subject, found, than,..."
3,A19FKU6JZQ2ECJ,B00005N7OJ,this beautiful magazine is in itself a work of...,"{now, many, it, discovered, information, ago, ..."
4,A25MDGOMZ2GALN,B00005N7P0,a great read every issue,"{read, issue, every, great, a}"


In [6]:
df_Sim['Similarity'] = 0
for index, row in df_Sim.iterrows():
    currVector = input_set.union(row['tokenReview'])
    l1 =[];l2 =[]
    for w in currVector:
        if w in input_set: l1.append(1)
        else: l1.append(0)
        if w in row['tokenReview']: l2.append(1)
        else: l2.append(0)
    c = 0
    for i in range(len(currVector)):
        c+= l1[i]*l2[i]
    if sum(l2) == 0:
        df_Sim.loc[index, 'Similarity'] = 0
    else:
        df_Sim.loc[index, 'Similarity'] = c / float((sum(l1)*sum(l2))**0.5)
df_Sim.head()

,reviewerID,asin,reviewText,tokenReview,Similarity
0,AH2IFH762VY5U,B00005N7P0,for computer enthusiast maxpc is a welcome sig...,"{own, wait, than, blessed, info, consider, rea...",0.146176
1,AOSFI0JEYU4XM,B00005N7P0,thank god this is not a ziff davis publication...,"{davis, time, something, if, bad, actually, go...",0.144673
2,A3JPFWKS83R49V,B00005N7OJ,antiques magazine is a publication made for an...,"{means, certainly, like, subject, found, than,...",0.178885
3,A19FKU6JZQ2ECJ,B00005N7OJ,this beautiful magazine is in itself a work of...,"{now, many, it, discovered, information, ago, ...",0.300000
4,A25MDGOMZ2GALN,B00005N7P0,a great read every issue,"{read, issue, every, great, a}",0.141421


In [7]:
df_Sim.sort_values(by=['Similarity'], ascending=False)

,reviewerID,asin,reviewText,tokenReview,Similarity
73095,A186HTO2DI1809,B00YQH9856,this magazine is very interesting and informative,"{interesting, this, and, informative, is, maga...",0.717137
15372,A1IBJDINIBX9EY,B00005N7QG,love this magazine and the price is great,"{this, and, is, great, price, magazine, the, l...",0.670820
37269,A3TWWC2B55AGVR,B00007B10Y,wonderful magazine and very great stories,"{and, great, wonderful, magazine, stories, very}",0.645497
81565,A3KVMN9OTVVUJR,B000BYD9V2,great magazine very interesting and informative,"{interesting, and, informative, great, magazin...",0.645497
67029,A3RO06TXVLAJU5,B005DNA3LU,the magazine is colorful and interesting,"{interesting, and, is, magazine, the, colorful}",0.645497
...,...,...,...,...,...
80244,A23QGG90CNKFRP,B00008WETF,as advertised,"{advertised, as}",0.000000
62525,AQTAZTY0T3LIF,B001C4Q06G,i have yet to receive an issue,"{issue, yet, have, receive, to, i, an}",0.000000
62526,A2E0P8AUHQOSGR,B001C4Q06G,thanks,{thanks},0.000000
35809,A1L5PVBLJJME7V,B00007B10Y,always enjoy,"{enjoy, always}",0.000000
